
# Modelo Predictivo de Riesgo Cardiovascular (Estudio Framingham)

Este notebook implementa un modelo de regresión logística para predecir el riesgo de un evento cardiovascular a 10 años, utilizando datos clínicos del estudio Framingham. Se incluye preprocesamiento, entrenamiento, evaluación y visualización de resultados.

## 1. Importar Librerías

```python
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
```

## 2. Cargar y Preprocesar Datos

```python
df = pd.read_csv("heart_disease.csv")

# Imputar valores faltantes
median_impute = ['glucose', 'totChol', 'cigsPerDay', 'BMI', 'heartRate']
for col in median_impute:
df[col].fillna(df[col].median(), inplace=True)

mode_impute = ['BPMeds', 'education']
for col in mode_impute:
df[col].fillna(df[col].mode()[0], inplace=True)
```

## 3. Normalización

```python
scaler = StandardScaler()
numeric_cols = ['age', 'cigsPerDay', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
```

## 4. Separación en variables predictoras y objetivo

```python
X = df.drop(columns='TenYearCHD')
y = df['TenYearCHD']
```

## 5. División en entrenamiento y prueba

```python
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
```

## 6. Entrenamiento del Modelo con `class_weight='balanced'`

```python
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train, y_train)
```

## 7. Evaluación del Modelo

```python
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_proba))
```

## 8. Curva ROC

```python
fpr, tpr, _ = roc_curve(y_test, y_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='AUC = {:.2f}'.format(roc_auc_score(y_test, y_proba)))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("Tasa de Falsos Positivos")
plt.ylabel("Tasa de Verdaderos Positivos")
plt.title("Curva ROC - Modelo Balanceado")
plt.legend()
plt.grid(True)
plt.show()
```

## 9. Interpretación de Coeficientes

```python
coef_df = pd.DataFrame({
'Variable': X.columns,
'Coeficiente': model.coef_[0],
'Interpretación': np.where(model.coef_[0] > 0, '↑ Aumenta riesgo', '↓ Disminuye riesgo')
}).sort_values(by='Coeficiente', ascending=False)

print(coef_df)
```

## 10. Conclusiones

- El modelo balanceado mejora la sensibilidad para detectar casos con riesgo cardiovascular.
- AUC = 0.70 indica una capacidad aceptable de discriminación.
- El análisis de coeficientes permite identificar variables con mayor peso en el riesgo, como edad, sexo, presión arterial, antecedentes de stroke y uso de medicamentos.
- Este modelo puede utilizarse como herramienta de apoyo clínico para priorizar seguimiento o intervención en pacientes con mayor riesgo estimado.

